In [3]:
%load_ext autoreload
%autoreload 2

In [73]:
from priceanalytics.data import download_df_map, cache_df_map, load_cached_df_map 
from priceanalytics.plot import MultiPlot
from priceanalytics import indicators as I
from priceanalytics.backtest import Backtester

from datetime import datetime, date, time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [5]:
tickers = ['AAPL', 'MSFT']
raw_tdfs = download_df_map(tickers, interval=1)
# cache_df_map(raw_tdfs, _dir="cache")


raw AAPL: 26711 datapoints from 2023-09-25 to 2023-11-14
raw MSFT: 22926 datapoints from 2023-09-25 to 2023-11-14


In [88]:
old_tdfs = load_cached_df_map(_dir="cache") # was @ interval = 5
old_dataset = old_tdfs['AAPL'] + old_tdfs['MSFT']

tickers = ['AAPL', 'MSFT', 'AMZN']
new_tdfs = download_df_map(tickers, interval=5)
new_dataset = []
for ticker, dfs in new_tdfs.items():
    new_dataset += dfs
    
dataset = old_dataset + new_dataset

datapts = sum([len(df) for df in dataset])
print("Loaded", len(dataset), "datasets containing", datapts, "datapoints")

raw AAPL: 6654 datapoints from 2023-09-25 to 2023-11-15
raw MSFT: 6086 datapoints from 2023-09-25 to 2023-11-15
raw AMZN: 6608 datapoints from 2023-09-25 to 2023-11-15
Loaded 129 datasets containing 23439 datapoints


In [118]:
import priceanalytics.backtest as bt
 
def test_and_strat(total, _len, sig_len, stop_loss=-10): #-1000% will never happen (hopefully)
    tpr = 1
    incrs = []
    
    for n, df in enumerate(total):

        df = I.add_andean_osc(df, _len=_len, sig_len=sig_len)
        df = df.iloc[_len:]

        with bt.Backtester(df, verbose=False) as b:
            last_buy = 0
            for i, fr in b:
                
                if b.crossover("andean_bull", "andean_bear"):
                    if not b.in_position: #should probably do this check automatically
                        b.buy()
                        last_buy = b.price()
                
                elif b.crossover("andean_signal", "andean_bull"):
                    if b.in_position: #should also probably do this check automatically
                        b.sell()
                    
                if b.in_position: # stop loss calculation
                    fallen = (b.price() - last_buy) / last_buy
                    if fallen < stop_loss:
                        b.sell()
            
            incr = b.results(1)
            incrs.append(incr)
            tpr *= (1 + incr)
    return tpr - 1, incrs


In [155]:
validation_tdfs = download_df_map(['GME'], interval=5)
validation_dataset = []
for ticker, dfs in validation_tdfs.items():
    validation_dataset += dfs
    
def calc_returns_homogenous(dataset):
    s = dataset[0].iloc[0]['close']
    e = dataset[-1].iloc[-1]['close']
    r = (e - s)/s
    print(f'homogenous dataset yields: {bt.perc_ret(r):5.2f}% returns')
    return r

ds_ret = calc_returns_homogenous(validation_dataset)

#tpr, incrs = test_and_strat(dataset, 14, 21)    
tpr, incrs = test_and_strat(validation_dataset, 14, 21, stop_loss=-.03)    



print('')
bt.print_test_results(incrs)


raw GME: 4377 datapoints from 2023-09-25 to 2023-11-15


IndexError: list index out of range

## notes

After a short time testing this out I noticed three things.
1. If neither line is above the signal line then it could be a choppy market.
2. If one line is almost on top of the signal line or barely above it then it could also be chop and sometimes it lasts a little while. Waiting until it is clearly above the signal line might work better.
3. If a position is taken and the bull/bear line goes below the signal line then it could be used as an exit. However it could also mean the market is going to have a bit of chop or slight retracement before continuing if the opposite line doesn't cross above the signal line.

So far it looks phenomenal

Oh I forgot to add, if in a position and the line stacks on top of the signal line then it can be an exit or everything else I listed in point 3 of my previous comment.